In [1]:
import sys; sys.path.append("../")
import pandas as pd

from clean.st_tammany_so_pprr import clean
from clean.st_tammany_so_cprr import clean as clean_cprr

In [2]:
pprr = clean()
pprr

../lib/clean.py:152: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\


,employee_id,last_name,first_name,birth_year,rank_code,rank_desc,pay_group,hire_year,hire_month,hire_day,term_year,term_month,term_day,uid
0,1,divincenti,frank,1923,CG01,school crossing guard,PSAF,1984,09,10,,,,6a22a802ff4f5072bdb2ef289c443e26
1,2,husser,linda,1949,EX04,consultant,AUX,2010,08,03,2016,06,30,af8f11a2cbd55d7e4de29ffaafffbe0d
2,3,jenkins,dickie,1955,MJ3,mj3 corr/comm/crt sec,AUX,1980,11,14,2006,02,03,70e5ff989ccf16676d48d668a08fcbd1
3,4,ranatza,gary,1964,M101,lieutenant protective services,PSAF,1996,09,26,2019,12,12,16e8d29524c6e31728c219dba5945aeb
4,5,kelly,robert,1926,CG01,school crossing guard,PSAF,1994,08,16,2001,10,31,3b029846b7b4037fb1ccf5a41ace73d0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,80996,beyer,justin,1974,RESV,reserve deputies,NaN,2020,01,14,,,,85f7e3e5d76557cc71dfbb8b84b56731
3118,80997,barberito,christopher,1990,RESV,reserve deputies,NaN,2020,01,28,,,,892431106b06f986346afabf739f6169
3119,80998,scarlato,michael,1980,RESV,reserve deputies,NaN,2020,02,13,,,,e937a1d07bbe01d1c467a04e6d985cf6
3120,80999,rogers,keith,1962,RESV,reserve deputies,NaN,2020,06,16,,,,84821a28cb1e0cec69382c7ec578f973


In [3]:
pprr[(pprr.first_name == 'charles') & (pprr.last_name.str.startswith('smith'))]

,employee_id,last_name,first_name,birth_year,rank_code,rank_desc,pay_group,hire_year,hire_month,hire_day,term_year,term_month,term_day,uid
116,119,smith jr,charles,1948,M202,corrections capt,AUX,1981,10,28,2002,04,11,854e9bbf9f908b776b5004c50eb61712
795,922,smith,charles,1963,A101,deputy/admin auxillary,AUX,2017,09,26,2019,12,18,9a8ad62809baf86d2ed47d49543a99d5
1376,1666,smith,charles,1962,P101,deputy,PSAF,2006,08,02,,,,ea1566bfddd6ffdb639ae513803e5c04


In [4]:
cprr = clean_cprr()
cprr

../clean/st_tammany_so_cprr.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.replace(r'(\d)\r\n(\d)', r'\1\2')\


,full_name,incident_date,dept_code,infraction,first_name,middle_name,last_name
0,Cody Talley,1/16/2015,36.0,DR.07:0200.400 - Maintenance and care of Vehi...,cody,,talley
1,Shawn Barrow,1/18/2015,20.0,DR.03:09.970.01 (a) - Job Performance - wanton...,shawn,,barrow
2,Kendall Twillie,2/7/2015,20.0,DR.03:09.934.01 - Failure to report - refusal ...,kendall,,twillie
3,Robert Jones,2/13/2015,12.0,DR.03:09.500.01 - disruptive or abusive behavi...,robert,,jones
4,Ryan Suhre,2/27/2015,36.0,SOP.1100:02.000- Property and Evidence Control...,ryan,,suhre
...,...,...,...,...,...,...,...
273,"Charles ""Chuck"" Smith","11/5,6,8,13,14,15,26/2019",20.0,DR.03:09.932.01 Failure to follow supervisor i...,charles,,smith
274,Jonathan Thornhill,12/19/2019,34.0,DR.03:09.957.01 Willful disregard of Agency ru...,jonathan,,thornhill
275,Timothy Scanlan,1/1/2020,34.0,DR. 03:09.977.01 Sleeping on duty,timothy,,scanlan
276,Alexandria Payne,Oct-Dec 2019,20.0,DR.03:09.916.01C Unsatifactory work performance,alexandria,,payne


In [5]:
# cprr.loc[250:260]
print(cprr.loc[273, 'infraction'])

DR.03:09.932.01 Failure to follow supervisor instructions, perform assigned work, or otherwise comply with policy and procedure. DR.03:09.933.01 Using rude, discourteous, racial related, or insulting language, or conduct offensive to the public, supervisors, or other STPSOemployees. DR.03:09.970.01 a. Wanton or willful neglect of duties. DR.03:09.957.01 Willful disregard of Agency rules, regulations, policies or procedures


In [6]:
cprr[~cprr.infraction.fillna('').str.contains(r'[a-zA-Z]{2,3}[^a-zA-Z]*\d{2,}[\d\:\.]+')]

,full_name,incident_date,dept_code,infraction,first_name,middle_name,last_name
15,Jonathan Sesser,3/17/2015,7.0,Warning - all technicians are remind via email...,jonathan,,sesser
21,Chase Sharp,5/31/2015,20.0,NaN,chase,,sharp
59,Suzanne Melara,11/17/2015,42.0,unsatisfactory work performance,suzanne,,melara
65,Christopher Boyll,11/30/2015,42.0,Unsatisfactory Work Performance - Warning,christopher,,boyll
112,Hattie Brown,7/1/2016,20.0,Due to background - discharged,hattie,,brown
124,Alex Loyd,8/12/2016,9.0,Excessive cell phone use - warning,alex,,loyd
138,Brian Harris,10/11/2016,20.0,Group 2 - Failure to act when necessary - sus...,brian,,harris
140,Kathy Levee,10/14/2016,20.0,Dy. Levee given written warning and not to con...,kathy,,levee
144,Justin Doles,10/26/2016,NaN,STPSO computers - warning,justin,,doles
232,Matthew Cleland,8/20/2018,26.0,verbal warning - making decisions of employees...,matthew,,cleland


In [7]:
cprr[~cprr.infraction.fillna('').str.contains(r'[a-zA-Z]{2,3}[^a-zA-Z]*\:[\d\:\.]+')]

,full_name,incident_date,dept_code,infraction,first_name,middle_name,last_name
15,Jonathan Sesser,3/17/2015,7.0,Warning - all technicians are remind via email...,jonathan,,sesser
16,Brandon Landry,3/28/2015,20.0,DR.03.08.150 a - An employee must always consc...,brandon,,landry
17,Derrick Torregano Jr,4/27/2015,26.0,Group 3 - DR.03.99.955.01 d. engaging in any c...,derrick,,torregano jr
21,Chase Sharp,5/31/2015,20.0,NaN,chase,,sharp
34,Darrylynn Washington,4/14/2015,28.0,DR.03.10.201 - A newly appointed employee may ...,darrylynn,,washington
59,Suzanne Melara,11/17/2015,42.0,unsatisfactory work performance,suzanne,,melara
65,Christopher Boyll,11/30/2015,42.0,Unsatisfactory Work Performance - Warning,christopher,,boyll
94,Hope Touzet,4/18/2016,9.0,SOP.0085.01.010.01 - Cash balance procedure - ...,hope,,touzet
112,Hattie Brown,7/1/2016,20.0,Due to background - discharged,hattie,,brown
116,Alyssa Shubert,7/18/2016,9.0,SOP.0085.01.010.01 - Cash balance procedure - ...,alyssa,,shubert


In [8]:
[print(x) for x in cprr.infraction]

 DR.07:0200.400 - Maintenance and care of Vehicles - Preventative maintenance in its entirety and adhere to all safety rules - suspension 80 hours
DR.03:09.970.01 (a) - Job Performance - wanton or willful neglect of duties - (b) manifesting cowardice, feigning illness, or otherwise attempting to shirk official duty.  DR.03:09.955.01 - Group 3 (f.) Taking any action which will impair the efficiency or reputation of the Agency, its members or employees. - suspension - 40 hours
DR.03:09.934.01 - Failure to report - refusal to work overtime upon request - suspension - 8 hours
DR.03:09.500.01 - disruptive or abusive behavior, insubordination, violation of safety rules; any act impairing the efficiency or reputation of the agency, its members or deputies - Grp 1, DR.03:09.915.01 a- Interfering with the work of others, DR.03:09.920.01 - Failure to follow safety rules and procedures - Grp 2 - DR.03:09.932.01 - Failure to follow supervisor instructions, perform assigned work, or otherwise compl

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,